# Bond valuation


## 1 — Imports
Import necessary packages: QuantLib (for dates), NumPy, SciPy interp1d, pandas.

In [1]:
import QuantLib as ql
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import math


## 2 — Dates setup
Define trade, settlement and maturity dates used for the bond schedule.

In [2]:
calendar = ql.WeekendsOnly()
trade_date = ql.Date(15, 8, 2025)
settlement_date = calendar.advance(trade_date, ql.Period('3D'), ql.Following)
maturity_date = ql.Date(15, 8, 2028)
print('Trade:', trade_date, 'Settle:', settlement_date, 'Maturity:', maturity_date)

Trade: August 15th, 2025 Settle: August 20th, 2025 Maturity: August 15th, 2028


## 3 — Zero rates input
Provide an array of observation times (years) and base zero rates (continuously compounded).

In [3]:
t = np.array([0, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0, 3.3])
zero_rates_base = np.array([0.0, 0.0673, 0.0855, 0.0926, 0.0971, 0.1005,
                            0.1035, 0.1063, 0.1089, 0.1115, 0.1142, 0.1169])
print('t:', t)
print('zero_rates_base:', zero_rates_base)

t: [0.  0.3 0.6 0.9 1.2 1.5 1.8 2.1 2.4 2.7 3.  3.3]
zero_rates_base: [0.     0.0673 0.0855 0.0926 0.0971 0.1005 0.1035 0.1063 0.1089 0.1115
 0.1142 0.1169]


### Convert zero rates to discount factors
DF(t) = exp(-r(t) * t). Note DF(0)=1.

In [4]:
discount_factor_base = np.exp(-zero_rates_base * t)
print('discount_factor_base:', np.round(discount_factor_base, 6))

discount_factor_base: [1.       0.980012 0.949994 0.920038 0.890012 0.860063 0.830025 0.799931
 0.770004 0.740041 0.709922 0.679927]


## 4 — Bond cashflow schedule
We build a semi-annual schedule and compute cashflow times (year fractions from trade date).

In [5]:
schedule = ql.MakeSchedule(trade_date, maturity_date, ql.Period('6M'), calendar=calendar, convention=ql.Following)
dates = list(schedule)
year_frac = np.array([(d - trade_date) / 365.0 for d in dates])
print('Payment dates (first 6):', dates[:6])
print('Year fractions (first 6):', np.round(year_frac[:6],6))

Payment dates (first 6): [Date(15,8,2025), Date(16,2,2026), Date(17,8,2026), Date(15,2,2027), Date(16,8,2027), Date(15,2,2028)]
Year fractions (first 6): [0.       0.506849 1.005479 1.50411  2.00274  2.50411 ]


### Cashflows and coupons
Assume notional and coupon. We'll build the fixed cashflows (semi-annual) and include principal on final paydown.

In [6]:
notional = 1_000_000
coupon_rate = 0.13  # annual coupon
# build cashflows per schedule
fixed_cashflows = [0.0]
for i in range(1, len(dates)-1):
    fixed_cashflows.append(notional * coupon_rate / 2.0)
fixed_cashflows.append(notional * coupon_rate / 2.0 + notional)
fixed_cashflows = np.array(fixed_cashflows)
print('Fixed cashflows (first 6):', fixed_cashflows[:6])

Fixed cashflows (first 6): [    0. 65000. 65000. 65000. 65000. 65000.]


### Interpolate discount factors to cashflow times
Use linear interpolation on DF vs t to get DF at payment times.

In [7]:
from scipy.interpolate import interp1d
interp_df_func = interp1d(t, discount_factor_base, kind='linear', fill_value='extrapolate')
df_at_pay = interp_df_func(year_frac)
print('Discount factors at pay dates:', np.round(df_at_pay,6))

Discount factors at pay dates: [1.       0.959315 0.909481 0.859651 0.809687 0.759606 0.709648]


### Present value calculation
PV = sum(cashflow_i * DF(t_i)). Compute PV of bond at trade date.

In [8]:
pv_base = float(np.sum(fixed_cashflows * df_at_pay))
print('Bond PV (base):', pv_base)

Bond PV (base): 1035128.4132108019


### Cashflow table (base scenario)
Build a pandas DataFrame listing each payment date, year fraction, cashflow, DF and PV of cashflow.

In [9]:
cf_base = pd.DataFrame({
    'Date': dates,
    'YearFrac': year_frac,
    'Cashflow': fixed_cashflows,
    'DF': df_at_pay,
})
cf_base['PV'] = cf_base['Cashflow'] * cf_base['DF']
cf_base['NetCF'] = cf_base['Cashflow']
cf_base['PV'].sum(), cf_base.head(12)

(1035128.4132108019,
                   Date  YearFrac   Cashflow        DF             PV  \
 0    August 15th, 2025  0.000000        0.0  1.000000       0.000000   
 1  February 16th, 2026  0.506849    65000.0  0.959315   62355.445291   
 2    August 17th, 2026  1.005479    65000.0  0.909481   59116.277982   
 3  February 15th, 2027  1.504110    65000.0  0.859651   55877.328398   
 4    August 16th, 2027  2.002740    65000.0  0.809687   52629.671320   
 5  February 15th, 2028  2.504110    65000.0  0.759606   49374.363016   
 6    August 15th, 2028  3.002740  1065000.0  0.709648  755775.327204   
 
        NetCF  
 0        0.0  
 1    65000.0  
 2    65000.0  
 3    65000.0  
 4    65000.0  
 5    65000.0  
 6  1065000.0  )

## 5 — bond valuation from zero rates
We write a small function that accepts zero rates and returns PV and cashflow table (using the same schedule).

In [22]:
def bond_valuation_from_zero(zero_rates):
    df = np.exp(-zero_rates * t)
    interp_func = interp1d(t, df, kind='linear', fill_value='extrapolate')
    df_pay = interp_func(year_frac)
    pv = float(np.sum(fixed_cashflows * df_pay))
    cf = pd.DataFrame({'Date': dates, 'YearFrac': year_frac, 'Cashflow': fixed_cashflows, 'DF': df_pay})
    cf['PV'] = cf['Cashflow'] * cf['DF']
    return pv, cf

# sanity check
pv_check, cf_check = bond_valuation_from_zero(zero_rates_base)
print('PV check equals pv_base?', round(pv_check,2), round(pv_base,2))

PV check equals pv_base? 1035128.41 1035128.41


## 6 — Stress scenarios
We'll implement multiple stress types:
- Parallel shifts (±50bps, ±100bps)
- Tenor bumps (bump short end, bump long end)
- Curve steepening/flattening (twists)
- Random Gaussian shocks to each zero rate
- Scaling (multiply rates by factor)

We'll compute PV for each scenario and compare to base.

In [11]:
# Parallel shifts
shifts = [0.005, 0.01, -0.005, -0.01]  # in decimals (50bps,100bps, -50bps, -100bps)
scenarios = []
for s in shifts:
    zr = zero_rates_base + s
    pv, _ = bond_valuation_from_zero(zr)
    scenarios.append({'Scenario': f'Parallel {s:+.3%}', 'zero_rates': zr, 'PV': pv})

print('Parallel shifts done')

Parallel shifts done


### Tenor bumps
Bump only short-end (first 3 rates) and long-end (last 3 rates) to see local sensitivity.

In [12]:
zr_short_bump = zero_rates_base.copy()
zr_short_bump[:3] += 0.01  # +100bps short
pv_short, _ = bond_valuation_from_zero(zr_short_bump)
scenarios.append({'Scenario': 'Short-end +100bps', 'zero_rates': zr_short_bump, 'PV': pv_short})

zr_long_bump = zero_rates_base.copy()
zr_long_bump[-3:] += 0.01  # +100bps long
pv_long, _ = bond_valuation_from_zero(zr_long_bump)
scenarios.append({'Scenario': 'Long-end +100bps', 'zero_rates': zr_long_bump, 'PV': pv_long})

print('Tenor bumps done')

Tenor bumps done


### Curve twist: steepen and flatten
Implement a steepening (short rates down, long rates up) and flattening (short up, long down).

In [13]:
zr_steepen = zero_rates_base.copy()
zr_steepen[:4] -= 0.005
zr_steepen[-4:] += 0.005
pv_steepen, _ = bond_valuation_from_zero(zr_steepen)
scenarios.append({'Scenario': 'Steepen +/-5bps', 'zero_rates': zr_steepen, 'PV': pv_steepen})

zr_flatten = zero_rates_base.copy()
zr_flatten[:4] += 0.005
zr_flatten[-4:] -= 0.005
pv_flatten, _ = bond_valuation_from_zero(zr_flatten)
scenarios.append({'Scenario': 'Flatten +/-5bps', 'zero_rates': zr_flatten, 'PV': pv_flatten})

print('Twist scenarios done')

Twist scenarios done


### Random Gaussian shocks
Add small Gaussian noise to each zero rate (mean 0, stdev 20bps) to simulate idiosyncratic movements.

In [14]:
np.random.seed(42)
rand_noise = np.random.normal(0.0, 0.002, size=zero_rates_base.shape)
zr_rand = zero_rates_base + rand_noise
pv_rand, _ = bond_valuation_from_zero(zr_rand)
scenarios.append({'Scenario': 'Random Gaussian (σ=20bps)', 'zero_rates': zr_rand, 'PV': pv_rand})
print('Random Gaussian shock done')

Random Gaussian shock done


### Scaling
Multiply the whole curve by a factor (e.g., +10% meaning rates increase by 10% multiplicatively).

In [15]:
zr_scale = zero_rates_base * 1.10
pv_scale, _ = bond_valuation_from_zero(zr_scale)
scenarios.append({'Scenario': 'Scale +10%', 'zero_rates': zr_scale, 'PV': pv_scale})
print('Scaling done')

Scaling done


### Large stress
A large parallel down shift (e.g., -500bps) to explore extreme scenario.

In [16]:
zr_large = zero_rates_base - 0.05
pv_large, _ = bond_valuation_from_zero(zr_large)
scenarios.append({'Scenario': 'Parallel -500bps', 'zero_rates': zr_large, 'PV': pv_large})
print('Large shift done')

Large shift done


## 7 — Build results DataFrame
Collect all scenario PVs, compute differences vs base and add descriptions.

In [17]:
results = pd.DataFrame(scenarios)
results['PV_base'] = pv_base
results['Diff_vs_Base'] = results['PV'] - pv_base
results['PctDiff_vs_Base'] = results['Diff_vs_Base'] / pv_base * 100.0

# add description
results['Description'] = results['Scenario']
results[['Scenario','PV','Diff_vs_Base','PctDiff_vs_Base','Description']]

,Scenario,PV,Diff_vs_Base,PctDiff_vs_Base,Description
0,Parallel +0.500%,1.021858e+06,-13270.777410,-1.282042,Parallel +0.500%
1,Parallel +1.000%,1.008773e+06,-26355.765755,-2.546135,Parallel +1.000%
2,Parallel -0.500%,1.048588e+06,13459.288460,1.300253,Parallel -0.500%
3,Parallel -1.000%,1.062238e+06,27109.850375,2.618984,Parallel -1.000%
4,Short-end +100bps,1.034814e+06,-313.941173,-0.030329,Short-end +100bps
5,Long-end +100bps,1.012328e+06,-22800.445315,-2.202668,Long-end +100bps
6,Steepen +/-5bps,1.023585e+06,-11542.970378,-1.115124,Steepen +/-5bps
7,Flatten +/-5bps,1.046851e+06,11722.296921,1.132449,Flatten +/-5bps
8,Random Gaussian (σ=20bps),1.036956e+06,1827.358068,0.176534,Random Gaussian (σ=20bps)
9,Scale +10%,1.005556e+06,-29572.379670,-2.856880,Scale +10%


### Display results table (summary)

In [18]:
pd.options.display.float_format = '{:,.2f}'.format
results_display = results[['Scenario','PV','Diff_vs_Base','PctDiff_vs_Base','Description']]
results_display

,Scenario,PV,Diff_vs_Base,PctDiff_vs_Base,Description
0,Parallel +0.500%,"1,021,857.64","-13,270.78",-1.28,Parallel +0.500%
1,Parallel +1.000%,"1,008,772.65","-26,355.77",-2.55,Parallel +1.000%
2,Parallel -0.500%,"1,048,587.70","13,459.29",1.30,Parallel -0.500%
3,Parallel -1.000%,"1,062,238.26","27,109.85",2.62,Parallel -1.000%
4,Short-end +100bps,"1,034,814.47",-313.94,-0.03,Short-end +100bps
5,Long-end +100bps,"1,012,327.97","-22,800.45",-2.20,Long-end +100bps
6,Steepen +/-5bps,"1,023,585.44","-11,542.97",-1.12,Steepen +/-5bps
7,Flatten +/-5bps,"1,046,850.71","11,722.30",1.13,Flatten +/-5bps
8,Random Gaussian (σ=20bps),"1,036,955.77","1,827.36",0.18,Random Gaussian (σ=20bps)
9,Scale +10%,"1,005,556.03","-29,572.38",-2.86,Scale +10%


## 8 — Per-period cashflow PV under one stress (Base and Parallel +100bps)
We'll compute and display cashflows table (DF and PV) for base and for a chosen stressed curve.

In [19]:
# cashflows for base
pv_base, cf_base = bond_valuation_from_zero(zero_rates_base)
# cashflows for parallel +100bps scenario (we created above as shifts[1])
zr_plus100 = zero_rates_base + 0.01
pv_p100, cf_p100 = bond_valuation_from_zero(zr_plus100)

print('PV base:', pv_base)
print('PV parallel +100bps:', pv_p100)

# show tables side by side
cf_compare = cf_base.copy()
cf_compare['DF_plus100'] = cf_p100['DF']
cf_compare['PV_plus100'] = cf_p100['PV']
cf_compare['PV_Diff'] = cf_compare['PV_plus100'] - cf_compare['PV']
cf_compare.head(12)

PV base: 1035128.4132108019
PV parallel +100bps: 1008772.6474555291


,Date,YearFrac,Cashflow,DF,PV,DF_plus100,PV_plus100,PV_Diff
0,"August 15th, 2025",0.00,0.00,1.00,0.00,1.00,0.00,0.00
1,"February 16th, 2026",0.51,"65,000.00",0.96,"62,355.45",0.95,"62,041.50",-313.94
2,"August 17th, 2026",1.01,"65,000.00",0.91,"59,116.28",0.90,"58,526.24",-590.04
3,"February 15th, 2027",1.50,"65,000.00",0.86,"55,877.33",0.85,"55,043.24",-834.09
4,"August 16th, 2027",2.00,"65,000.00",0.81,"52,629.67",0.79,"51,587.43","-1,042.24"
5,"February 15th, 2028",2.50,"65,000.00",0.76,"49,374.36",0.74,"48,154.67","-1,219.70"
6,"August 15th, 2028",3.00,"1,065,000.00",0.71,"755,775.33",0.69,"733,419.57","-22,355.76"
